In [ ]:
import numpy as np
from numpy.linalg import solve
from scipy.stats import poisson

n = 10                                 ##Capacidade do armazém de estoques
S = list(range(0,n+1))                 ##Espaço de estados
r = 4                                  ##Preço de venda do produto
c = 2                                  ##Custo de aquisição do produto
h = 0.5                                ##Custo de manter estoques
gamma = 0.9
demanda_media = 6                      ##Lambda da Poisson

##Função de transição de estado
def transicao(s,x,d):

    return np.maximum(s+x-d, 0)

##Função recompensa
def recompensa(s,x,d):

    return r*np.minimum(s+x,d)-c*x-h*np.maximum(s+x-d, 0)

def computa_recompensa_media(s,x):
    
    ##Computa o valor esperado da recompensa
    valor_esperado_recompensa = 0
        
    for d in range(0, s+x):
        valor_esperado_recompensa+= recompensa(s,x,d)*poisson.pmf(d,demanda_media)

    prob = 1
    for d in range(0, s+x):
        prob-=poisson.pmf(d,demanda_media)

        ##print("recompensa(s,x,d)", recompensa(s,x,s+x))

    valor_esperado_recompensa+=recompensa(s,x,s+x)*prob
  
    return valor_esperado_recompensa

def computa_valor_esperado_v_pol(s,x, v_pol):
    
    valor_esperado = 0
    for d in range(0, s+x):
        valor_esperado+= v_pol[transicao(s,x,d)]*poisson.pmf(d,demanda_media)
    
    prob = 1
    for d in range(0, s+x):
        prob-=poisson.pmf(d,demanda_media)


    valor_esperado += v_pol[0]*prob

    return valor_esperado

def constroi_matriz_P(pi, demanda_media):
    
    P = np.zeros((len(S), len(S)))

    for s in S:
       
        x = pi[s]    ##Decisão obtida com a política
       
        for j in range(0, s+x):
            P[s,s+x-j] = poisson.pmf(j, demanda_media)

        ##Computa a probabilidade da transição para s_linha = 0
        prob = 1
        for d in range(0, s+x):
            prob-=poisson.pmf(d,demanda_media)

        P[s,0] = prob

        ##Probabilidade de transição de s para s_linha > s+x é zero
        for j in range(s+x, len(S)):
           P[s,j] = 0


    return P

pi = np.zeros(len(S), dtype=np.int64)       ##Política inicial

##pi = np.array([4,3,2,1,0,0,0,0,0,0,0],dtype=np.int64)       ##Política inicial

##Loop da iteração de política
k = 0
while True:
    
    print(f"Iteração = {k}")
    print("Politica atual = ", pi)

    ##Avalia a política

    ##Constroi o vetor de recompensas
    rec = np.zeros(len(S))

    for s in S:
        
        ##Computa o valor esperado da recompensa
        
        x = pi[s]    ##Decisão obtida com a política

        valor_esperado_recompensa = computa_recompensa_media(s,x)
        rec[s] = valor_esperado_recompensa
        
    ##Constroi a matriz de transição
    P = constroi_matriz_P(pi, demanda_media)


    ##COmputa o valor da política
    I = np.eye(len(S))    ##Matriz identidade
    A = (I-gamma*P)
    v_pol = solve(A, rec)

    print("Valor da politica = ", v_pol)

    ##Melhoria de política

    nova_pi = np.zeros(len(S), dtype=np.int64)
     
    for s in S:
        
        ##Geração conjunto de decisões viáveis
        Xs = list(range(0, n-s+1))
        valor_melhor_decisao = -np.inf

        for x in Xs:
            
            valor_decisao = computa_recompensa_media(s,x)+gamma*computa_valor_esperado_v_pol(s,x, v_pol)
        
            if valor_decisao > valor_melhor_decisao:
                valor_melhor_decisao = valor_decisao
                melhor_decisao = x

        nova_pi[s] = melhor_decisao

    print("Nova politica = ", nova_pi)

    if (nova_pi-pi).sum() == 0:
        pi = nova_pi
        break
    
    else:
        pi = nova_pi
        k+=1

print("\nMelhor política obtida:")
for s in S:
  print(f"Pi({s}) = {pi[s]}")

Iteração = 0
Politica atual =  [0 0 0 0 0 0 0 0 0 0 0]
Valor da politica =  [ 0.          3.98884562  7.96415666 11.89868285 15.75091223 19.47750423
 23.04803056 26.45177032 29.69386258 32.78598621 35.73839942]
Nova politica =  [10  9  8  7  6  5  4  3  2  1  0]
Iteração = 1
Politica atual =  [10  9  8  7  6  5  4  3  2  1  0]
Valor da politica =  [ 87.51346548  89.51346548  91.51346548  93.51346548  95.51346548
  97.51346548  99.51346548 101.51346548 103.51346548 105.51346548
 107.51346548]
Nova politica =  [7 6 5 4 3 2 1 0 0 0 0]
Iteração = 2
Politica atual =  [7 6 5 4 3 2 1 0 0 0 0]
Valor da politica =  [ 95.17335285  97.17335285  99.17335285 101.17335285 103.17335285
 105.17335285 107.17335285 109.17335285 111.16014574 112.86796598
 114.38565782]
Nova politica =  [7 6 5 4 3 2 1 0 0 0 0]
Melhor politica =  [7 6 5 4 3 2 1 0 0 0 0]
